In [ ]:
import awkward
import numpy as np
from coffea import nanoevents

def get_array(array):
    return array.compute() if nanoevents_mode == "dask" else array

nanoevents_mode = "dask"

events = nanoevents\
         .NanoEventsFactory\
         .from_root({'../tests/samples/nano_dy.root': "Events"}, mode=nanoevents_mode)\
         .events()

def some_event_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]

events.add_systematic("RenFactScale", "UpDownSystematic", "weight", some_event_weight)
events.add_systematic("XSectionUncertainty", "UpDownSystematic", "weight", some_event_weight)

muons = events.Muon
jets = events.Jet

In [ ]:
def muon_pt_scale(pt):   
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * pt[:, None]

def muon_pt_resolution(pt):   
    return np.random.normal(pt[:,None], np.array([0.02, 0.01], dtype=np.float32))

def muon_eff_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]

muons.add_systematic("PtScale", "UpDownSystematic", "pt", muon_pt_scale)
muons.add_systematic("PtResolution", "UpDownSystematic", "pt", muon_pt_resolution)
muons.add_systematic("EfficiencySF", "UpDownSystematic", "weight", muon_eff_weight)

def jet_pt_scale(pt):   
    return (1.0 + np.array([0.10, -0.10], dtype=np.float32)) * pt[:, None]

def jet_pt_resolution(pt):   
    return np.random.normal(pt[:,None], np.array([0.20, 0.10], dtype=np.float32))

jets.add_systematic("PtScale", "UpDownSystematic", "pt", jet_pt_scale)
jets.add_systematic("PtResolution", "UpDownSystematic", "pt", jet_pt_resolution)

In [ ]:
muon_pt = awkward.flatten(muons.pt)
get_array(muon_pt)

In [ ]:
muon_PtScale_up = awkward.flatten(muons.systematics.PtScale.up)
get_array(muon_PtScale_up)

In [ ]:
muon_PtScale_up_pt = awkward.flatten(muons.systematics.PtScale.up.pt)
get_array(muon_PtScale_up_pt)

In [ ]:
jets_pt = awkward.flatten(jets.pt)
get_array(jets_pt)

In [ ]:
jets_PtScale_up = awkward.flatten(jets.systematics.PtScale.up)
get_array(jets_PtScale_up)

In [ ]:
jets_PtScale_up_pt = awkward.flatten(jets.systematics.PtScale.up.pt)
get_array(jets_PtScale_up_pt)

In [ ]:
renfact_up = events.systematics.RenFactScale.up.weight_RenFactScale
get_array(renfact_up)

In [ ]:
# TODO: Make it so that syst_muons.Y > X returns boolean values
#       for all variations over Y. 
#       Requires some tracking of (pieces of) "what".